In [1]:
import os
import sys

In [2]:
IN_COLAB='google.colab' in sys.modules

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/trabalho_final_UNICAMP-IR/artigo/unicamp_IR"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    TREC_EVAL="/content/drive/MyDrive/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval"
else:
    WORKING_FOLDER="/media/eduseiti/bigdata01/unicamp/ia368v_dd/unicamp_IR/"
    TREC_EVAL="/media/eduseiti/bigdata01/unicamp/ia368v_dd/pyserini/tools/eval/trec_eval.9.0.4/trec_eval"

In [4]:
import pandas as pd
import numpy as np

In [5]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [14]:
ALL_EVALUATIONS="tests/evaluation_16_retrievers_000_009_sampled_50_queries_gpt-4-1106-preview_20240125.tsv"

BM25_RUN="tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt"
BM25_E5_RUN="tests/bm25_e5_docs_000_999_50_sampled_queries_run.txt"
BM25_MT5_RUN="tests/200-queries-passagens-by-BM25-mT5_v4_with_query_id_run.txt"
SPLADE_RUN="tests/splade_pt_result_with_clueweb_id_run.txt"
COLBERTX_RUN="tests/colbertx_all_znormalized_validation_scores_000_199_run.txt"

E5_RUN="tests/e5_docs_000_999_50_sampled_queries_run.txt"
E5_BASE_RUN="tests/e5_base_docs_000_999_50_sampled_queries_run.txt"
SPLADE_MT5_RUN="tests/splade_mt5_docs_000_099_50_sampled_queries_run.txt"

BM25_1M_RUN="tests/bm25_1M_docs_000_999_sampled_50_queries_run.txt"
BM25_1M_MASSIVEWEB_RUN="tests/bm25_massiveweb_1M_docs_000_999_sampled_50_queries_run.txt"

COLBERTX_1M_MASSIVEWEB_RUN="tests/colbertx_all_retrieval_1M_dataset_20240105_run.txt"

E5_COLBERTX_RRF_RUN="tests/colbertx_e5_RRF_000_999_sampled_50_queries_run.txt"
E5_SPLADE_RRF_RUN="tests/splade_e5_fixed_RRF_000_999_sampled_50_queries_run.txt"

ADAV2_RUN="tests/adav2_1M_docs_000_999_sampled_50_queries_run.txt"

In [7]:
BM25_MT5_1M_MASSIVEWEB="tests/bm25_mt5_1M_docs_000_099_sampled_50_queries.tsv"

In [8]:
DATASET_1M_FIELDS=['bm25_1M_massiveweb', 'colbertx_1M_massiveweb', 'adav2', 'bm25_mT5_1M_massiveweb']

In [9]:
def prepare_run_format(results_filename, add_index=True, score_field='score', invert_score=False, retriever_name="retriever"):

    results_df = pd.read_csv(results_filename, sep='\t')

    display(results_df.head())

    results_grouped_df = results_df.groupby("query_id")['passage_id'].count()

    how_many_passages_per_question = results_grouped_df.iloc[0]

    if invert_score:
        print("Inverting score values in '{}' field...".format(score_field))

        results_df['score_inv'] = 1 - results_df[score_field]
        score_field = 'score_inv'

    run_df = results_df.sort_values(["query_id", score_field], ascending=[True, False])

    run_df['Q0'] = "Q0"
    run_df['comment'] = retriever_name

    if add_index:
        run_df['index'] = np.tile(list(range(1, how_many_passages_per_question + 1)), results_grouped_df.shape[0])

    output_filename = "{}_run.txt".format(os.path.splitext(results_filename)[0])

    print("output_filename={}".format(output_filename))

    run_df[['query_id', 'Q0', "passage_id", "index", score_field, "comment"]].to_csv(output_filename, header=None, index=False, sep=" ")

    return output_filename, run_df

In [10]:
retrievers_results = []

In [52]:
retrievers_1M_results = []

### Format GPT-4 evaluations to qrels format

In [15]:
qrels_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))
qrels_1M_filename = os.path.join(os.path.dirname(ALL_EVALUATIONS), "{}_1M_qrel.txt".format(os.path.splitext(os.path.basename(ALL_EVALUATIONS))[0]))

In [16]:
if os.path.exists(qrels_filename) == False:
    eval_df = pd.read_csv(ALL_EVALUATIONS, sep='\t')

    eval_df

    eval_df.drop_duplicates("query_id")[['query_id', 'query']]

    #### Check scores distribution on the current qrels

    np.unique(eval_df['score'].to_numpy(), return_counts=True)

    for group_name, group_df in eval_df.groupby('query_id'):
        print("\nquery_id={}".format(group_name))

        print(np.unique(group_df['score'].to_numpy(), return_counts=True))

    eval_df.groupby('query_id').count()

    eval_df.groupby('query_id').count()['query'].describe()

    #### Prepare the qrels format for trec_eval tools

    eval_df['0'] = '0'

    print("\n\n")
    print(qrels_filename)
    print(qrels_1M_filename)

    eval_df.sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_filename, header=None, index=False, sep=" ")


    #### Select 1M dataset evaluations

    eval_df[eval_df[DATASET_1M_FIELDS].sum(axis=1) > 0].sort_values('query_id')[['query_id', '0', 'passage_id', 'score']].to_csv(qrels_1M_filename, header=None, index=False, sep=" ")

else:
    print("QRELS files ({}) already created...".format(qrels_filename))


query_id=1
(array([0, 1, 2, 3]), array([21, 42, 16, 10]))

query_id=2
(array([0, 1, 2, 3]), array([96,  2,  1,  2]))

query_id=9
(array([0, 1, 2, 3]), array([61, 10, 12,  5]))

query_id=11
(array([0, 1, 2, 3]), array([51, 17,  8,  2]))

query_id=13
(array([0, 1, 2, 3]), array([40, 14, 22,  4]))

query_id=15
(array([0, 1, 2, 3]), array([31, 20,  4, 17]))

query_id=17
(array([0, 1, 2, 3]), array([29, 10,  2, 36]))

query_id=20
(array([0, 1, 2, 3]), array([18, 57, 26,  7]))

query_id=21
(array([0, 1, 2, 3]), array([53, 21,  8, 19]))

query_id=22
(array([0, 1, 2, 3]), array([58, 15,  2,  7]))

query_id=24
(array([0, 1, 2, 3]), array([61, 31, 11,  2]))

query_id=26
(array([0, 1, 2, 3]), array([14, 17, 14, 33]))

query_id=28
(array([0, 1, 2, 3]), array([60, 16, 13,  4]))

query_id=47
(array([0, 1, 2, 3]), array([51, 31, 29,  1]))

query_id=49
(array([0, 1, 2, 3]), array([28, 29, 24,  1]))

query_id=51
(array([0, 1, 2, 3]), array([39, 21, 12,  1]))

query_id=54
(array([0, 1, 2, 3]), array([ 

## 10M dataset

### Compute BM25 nDCG@10

In [17]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} tests/200-queries-passagens-by-BM25-pt_v2_with_query_id_run.txt

In [18]:
retrievers_results.append({'retriever':'BM25', 'nDCG@10': result[0].split('\t')[-1]})

In [19]:
retrievers_results

[{'retriever': 'BM25', 'nDCG@10': '0.4504'}]

### Compute BM25 + E5 nDCG@10

In [20]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_E5_RUN}

In [21]:
retrievers_results.append({'retriever':'BM25+E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 nDCG@10

In [22]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_MT5_RUN}

In [23]:
retrievers_results.append({'retriever':'BM25+mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE nDCG@10

In [24]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_RUN}

In [25]:
retrievers_results.append({'retriever':'SPLADEv2_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERTX nDCG@10

In [26]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_RUN}

In [27]:
retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 nDCG@10

In [28]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_RUN}

In [29]:
retrievers_results.append({'retriever':'E5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 base nDCG@10

In [30]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_BASE_RUN}

In [31]:
retrievers_results.append({'retriever':'E5 base', 'nDCG@10': result[0].split('\t')[-1]})

### Compute SPLADE + mT5 nDCG@10

In [32]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {SPLADE_MT5_RUN}

In [33]:
retrievers_results.append({'retriever':'SPLADEv2_pt-BR + mT5', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 + ColBERT-X RRF nDCG@10

In [34]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_COLBERTX_RRF_RUN}

In [35]:
retrievers_results.append({'retriever':'E5 + ColBERT-X mMARCO_pt-BR RRF', 'nDCG@10': result[0].split('\t')[-1]})

### Compute E5 + SPLADE RRF nDCG@10

In [36]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {E5_SPLADE_RRF_RUN}

In [37]:
retrievers_results.append({'retriever':'E5 + SPLADEv2_pt-BR RRF', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M nDCG@10

In [38]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_RUN}

retrievers_results.append({'retriever':'BM25 1M', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 1M massiveweb filtered nDCG@10

In [39]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {BM25_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERT-X 1M massiveweb filtered nDCG@10

In [40]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {COLBERTX_1M_MASSIVEWEB_RUN}

retrievers_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ADAv2 1M massiveweb filtered nDCG@10

In [41]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {ADAV2_RUN}

retrievers_results.append({'retriever':'ADA v2 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Prepare BM25 + mT5 1M massiveweb filtered run and compute nDCG@10

In [42]:
bm25_mt5_filename, _ = prepare_run_format(BM25_MT5_1M_MASSIVEWEB, retriever_name="BM25+mT5")

,query,colbertx_id,passage,query_id,passage_id,score
0,Onde está localizada a Praça XV de Novembro?,3189242,"Endereço completo: Rua XV de Novembro, 759 2º Piso salas 40 A 46-Centro, Blumenau-SC, CEP :89010902. Importante: As informações dessa página tem como fonte o Portal Brasileiro de Dados Abertos, com isso não é possível garantir que todos os dados estejam 100% atualizados e corretos, desde a data da última atualização da base de dados do Portal Brasileiro de Dados Abertos",105,clueweb22-pt0000-67-03241_1,1000
1,Onde está localizada a Praça XV de Novembro?,7749553,"Uma boa maneira de entrar em contato com os moradores e também com a cidade é percorrer a Rua XV de Novembro. Ela é a principal avenida de Blumenau e está repleta de comércios e lojas com produtos típicos da região (especialmente produtos com a temática da Oktoberfest). Ao caminhar, não deixe de reparar nas casas em estilo enxaimel. Apesar de quase nenhuma delas ser original (infelizmente é verdade), elas formam belos cenários para fotos. As mais chamativas da rua são o Castelinho — uma grande casa onde funciona a loja Havan — e o prédio da Prefeitura de Blumenau, onde uma locomotiva de época e um relógio de flores ajudam a compor a cena. Se quiser mesmo ver uma construção original, procure a pequena casa com anões na sacada. Ainda na Rua XV de Novembro, não deixe de visitar a Catedral São Paulo, a Fundação Cultural São Paulo e o Biergarten, um charmoso jardim de frente para o rio Itajaí-Açu.",105,clueweb22-pt0001-61-06106_2,990
2,Onde está localizada a Praça XV de Novembro?,8799939,"Com a inauguração da “PARADA XV”, começaram a chegar outras famílias fixando-se na parte nova, ou seja, Vila 15 de Novembro, Artur e Da, Elisa Zenker, Carlos Stafen, Carlos Czizek, Skrupsen, Júlio Navega e outros. Da. Elisa Zenker era originária de recursos abastados, muito caridosa e social, procurava auxiliar a todos que batiam e sua porta, promovia grandes festas sociais, sua visita faziam a alegria da criançada com suas possantes motocicletas. Foi, com outras pessoas, a fundadora de um clube com o nome de “Eldorado” sendo este o clube social da época, hoje na saudade e lembrança daqueles que tiveram a oportunidade de conhecer e participar de suas reuniões. Foi Da. Elisa que, com seus recursos, montou um pequeno Parque Infantil onde hoje está localizada a Praça XV de Novembro e que servia para divertimento das crianças. Parada XV de Novembro teve uma pequena indústria de fogos, fundada por seu Dante e, mais tarde, a família Setti, fechando-se por volta de 1931.",105,clueweb22-pt0001-84-16421_6,980
3,Onde está localizada a Praça XV de Novembro?,1066879,"Importância porque ali está o Monumento do Cooperativismo, simbolizando sua importância para a cidade, histórica e culturalmente. Esta praça fica na Avenida 15 de novembro, no centro da cidade, e conta ainda com um labirinto de arbustos, o labirinto verde. Localizado no coração da praça é uma das atrações que diverte e entretém as diferentes gerações que passam por lá. Para almoçar, experimente logo adiante o fogão a lenha do restaurante Unser Haus. E já pode dar uma visitada na cervejaria Edelbrau, visitando alguns dos melhores locais da cidade num só passeio. Mais adiante, pela 116, há também a comida caseira do restaurante Colina Verde, um verdadeiro banquete. E se der vontade de uma sobremesa, passe na Mukli Alfajores. Um uruguaio que se apaixonou pela cidade trouxe para Nova Petrópolis essas delícias de seu país. Além de poder levar para casa variados tipos de alfajor, aproveite e experimente o milk shake da casa. Se você está procurando o que fazer dentro da parte urbana de Nova Petrópolis, precisa passar pela praça. 10. Casa Amarela Que tal uma galeria de arte para você apreciar a importância histórica de Nova Petrópolis de acordo com o ponto de vista de quem ali viveu e ainda vive?",105,clueweb22-pt0000-22-03330_10,970
4,Onde está localizada a Praça XV de Novembro?,3795224,"Plaza Corporate - Hon Hon / Plaza Corporate Encont

output_filename=tests/bm25_mt5_1M_docs_000_099_sampled_50_queries_run.txt


In [43]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_filename} {bm25_mt5_filename}

retrievers_results.append({'retriever':'BM25+mT5 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [44]:
pd.DataFrame(retrievers_results)

,retriever,nDCG@10
0,BM25,0.4504
1,BM25+E5,0.6414
2,BM25+mT5,0.7164
3,SPLADEv2_pt-BR,0.5840
4,ColBERT-X_mMARCO_pt-BR,0.6322
5,E5,0.5608
6,E5 base,0.4001
7,SPLADEv2_pt-BR + mT5,0.7021
8,E5 + ColBERT-X mMARCO_pt-BR RRF,0.6430
9,E5 + SPLADEv2_pt-BR RRF,0.6331


## 1M dataset

### Compute BM25 1M massiveweb filtered nDCG@10

In [53]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {BM25_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'BM25 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ColBERT-X massiveweb filtered nDCG@10

In [54]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {COLBERTX_1M_MASSIVEWEB_RUN}

retrievers_1M_results.append({'retriever':'ColBERT-X_mMARCO_pt-BR 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute ADA v2 massiveweb filtered nDCG@10

In [55]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {ADAV2_RUN}

retrievers_1M_results.append({'retriever':'ADA v2 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Compute BM25 + mT5 1M massiveweb filtered nDCG@10

In [56]:
result = !{TREC_EVAL} -c -mndcg_cut.10 \
    {qrels_1M_filename} {bm25_mt5_filename}

retrievers_1M_results.append({'retriever':'BM25+mT5 1M massiveweb', 'nDCG@10': result[0].split('\t')[-1]})

### Check the overall results

In [57]:
pd.DataFrame(retrievers_1M_results)

,retriever,nDCG@10
0,BM25 1M massiveweb,0.4248
1,ColBERT-X_mMARCO_pt-BR 1M massiveweb,0.5169
2,ADA v2 1M massiveweb,0.5983
3,BM25+mT5 1M massiveweb,0.3424
